# Wind speed invertion

This notebook example show how to invert wind speed from sigma0, using models from LUT or GMF, using [xsarsea.windspeed.invert_from_model](../basic_api.rst#xsarsea.windspeed.invert_from_model)

In [ ]:
import xsarsea
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# optional debug messages
import logging
logging.basicConfig()
logging.getLogger('xsarsea.windspeed').setLevel(logging.DEBUG) # or .setLevel(logging.INFO)

## Read sarwing owi file

download more from https://cyclobs.ifremer.fr/static/sarwing_datarmor/processings/c39e79a/default/reports/shoc_dailyupdate_names/report.html (in the 'safe' column, download files named like `*-owi-xx-*.nc`)

In [ ]:
sarwing_owi_file = xsarsea.get_test_file('s1a-iw-owi-xx-20210909t130650-20210909t130715-039605-04AE83.nc')
sarwing_ds = xsarsea.read_sarwing_owi(sarwing_owi_file)
sarwing_ds

## Get ancillary wind

Ecmwf wind is stored in owi file in *geographical* (deg N/S) convention. `xsarsea.windspeed` need it relative to *sample* (ie antenna), as a complex number.

We use [xsarsea.dir_geo_to_sample](../basic_api.rst#xsarsea.dir_geo_to_sample) to convert `sarwing_ds.owiEcmwfWindDirection` (deg) to radians, relative to sample, using `sarwing_ds.owiHeading`


In [ ]:
owi_ecmwf_wind = sarwing_ds.owiEcmwfWindSpeed * np.exp(1j* xsarsea.dir_geo_to_sample(sarwing_ds.owiEcmwfWindDirection, sarwing_ds.owiHeading))
sarwing_ds = xr.merge([
    sarwing_ds,
    owi_ecmwf_wind.to_dataset(name='owi_ancillary_wind'),
])

## Ancillary wind control plot

### Directions in default convention

We check that ancillary wind is correct, by plotting the wind speed, the wind direction, and the wind direction components.

The *sample* component increase with *sample* axis, and *line* component increase with *line*.

Note that we have to use `kdims=['sample','line']` to properly display the vectorfield, because memory order `['sample','line']` is geographycally transposed 

In [ ]:
sub_sarwing_ds = sarwing_ds.isel(line=slice(None, None, 10), sample=slice(None, None, 10))

vectorfield = hv.VectorField(
    (
        sub_sarwing_ds.sample, sub_sarwing_ds.line,
        xsarsea.dir_geo_to_sample(sub_sarwing_ds.owiEcmwfWindDirection, sub_sarwing_ds.owiHeading),
        sub_sarwing_ds.owiEcmwfWindSpeed
    )
)

owi_ecmwf_wind_sample = np.real(owi_ecmwf_wind)
owi_ecmwf_wind_line = np.imag(owi_ecmwf_wind)
(
    hv.Image(sarwing_ds.owiEcmwfWindSpeed, kdims=['sample','line']).opts(title='speed and dir', clim=(0,50), cmap='jet') * vectorfield
    + hv.Image(owi_ecmwf_wind_sample, kdims=['sample','line']).opts(title='sample component',cmap='bwr') * vectorfield
    + hv.Image(owi_ecmwf_wind_line, kdims=['sample','line']).opts(title='line component',cmap='bwr') * vectorfield
).opts(title='Direction component in standard convention')

### Directions in gmf or lut convention

In gmf or lut, the sample wind direction component is **negative** if the wind vector is in the same direction as increasing sample, and line component is  **positive**  if if the wind vector is in the same direction as increasing line.

Switching to one convention to another can be done with `-np.conj(complex_dir)`

Note that in `xsarsea.windspeed`,  all directions are in **gmf or lut convention**

In [ ]:
(
    hv.Image(sarwing_ds.owiEcmwfWindSpeed, kdims=['sample','line']).opts(title='speed and dir', clim=(0,50), cmap='jet') * vectorfield
    + hv.Image(-owi_ecmwf_wind_sample, kdims=['sample','line']).opts(title='sample component',cmap='bwr') * vectorfield
    + hv.Image(owi_ecmwf_wind_line, kdims=['sample','line']).opts(title='line component',cmap='bwr') * vectorfield
).opts(title='Direction component in gmf or lut convention')

## Copol inversion

Copol wind inversion, using model and ancillary wind

In [ ]:
wind_co = windspeed.invert_from_model(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        ancillary_wind = -np.conj(sarwing_ds.owi_ancillary_wind),
        model='cmod5n')
windspeed_co = np.abs(wind_co)

## differences with sarwing

In [ ]:
windspeed_diff =np.abs(windspeed_co) - sarwing_ds.owiWindSpeed_Tab_copol
(
    (
        hv.Image(np.abs(windspeed_co), kdims=['sample','line']).opts(title='xsarsea') 
        + hv.Image(sarwing_ds.owiWindSpeed_Tab_copol, kdims=['sample','line']).opts(title='sarwing' )
    ).opts(hv.opts.Image(cmap='jet', clim=(0,50))) 
    + hv.Image(windspeed_diff, kdims=['sample','line']).opts(cmap='jet', clim=(-0.5,0.5), title='xsarsea-sarwing\nmean=%.4f std=%.4f' % (np.mean(windspeed_diff), np.std(windspeed_diff)))
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## Crosspol inversion

Sarwing crosspol inversion is done with model `cmodms1ahw`, that is only available from lut.

So we use [xsarsea.windspeed.register_all_nc_luts](../basic_api.rst#xsarsea.windspeed.register_all_nc_luts) to register this lut.


In [ ]:
nc_luts_path = xsarsea.get_test_file('xsarsea_luts')
windspeed.register_all_nc_luts(nc_luts_path)

To match sarwing outputs, we will flatten `sarwing_ds.owiNesz_cross` with [xsarsea.windspeed.nesz_flattening](../basic_api.rst#xsarsea.windspeed.nesz_flattening) , and compute `dsig_cr`

In [ ]:
# nesz cross flattening, and dsig_cr
nesz_cross_flat = windspeed.nesz_flattening(sarwing_ds.owiNesz_cross, sarwing_ds.owiIncidenceAngle)
dsig_cr = (1.25 / (sarwing_ds.owiNrcs_cross / nesz_cross_flat )) ** 4.

windspeed_cr = windspeed.invert_from_model(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs_cross,
        dsig_cr=dsig_cr,
        model='cmodms1ahw')
windspeed_cr = np.abs(windspeed_cr)

In [ ]:
windspeed_diff = windspeed_cr - sarwing_ds.owiWindSpeed_Tab_crosspol
(
    (hv.Image(windspeed_cr).opts(title='xsarsea') + hv.Image(sarwing_ds.owiWindSpeed_Tab_crosspol).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,50))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-0.2,0.2), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## Dualpol inversion

Dualpol give better wind inversion results.

Dualpol inversion also returns copol wind.

In [ ]:
sarwing_luts_subset_path = xsarsea.get_test_file('sarwing_luts_subset')
windspeed.register_all_sarwing_luts(sarwing_luts_subset_path)

wind_co, wind_dual = windspeed.invert_from_model(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        sarwing_ds.owiNrcs_cross,
        ancillary_wind=-np.conj(sarwing_ds.owi_ancillary_wind),
        dsig_cr = dsig_cr,
        model=('cmod5n','cmodms1ahw'))
windspeed_co = np.abs(wind_co)
windspeed_dual =  np.abs(wind_dual)

## differences with sarwing

In [ ]:
windspeed_diff = windspeed_dual - sarwing_ds.owiWindSpeed_Tab_dualpol_2steps
(
    (hv.Image(windspeed_dual).opts(title='xsarsea') + 
     hv.Image(sarwing_ds.owiWindSpeed_Tab_dualpol_2steps).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,50))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-0.5,0.5), title='xsarsea-sarwing\nmean=%.4f std=%.4f' % (np.mean(windspeed_diff), np.std(windspeed_diff)))
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## direction output

In [ ]:
sarwing_ds["winddir_dual"] = (90 - (np.angle(-np.conj(wind_dual),deg=True)) + sarwing_ds.owiHeading)%360
sub_sarwing_ds = sarwing_ds.isel(line=slice(None, None, 10), sample=slice(None, None, 10))

vectorfield = hv.VectorField(
    (
        sub_sarwing_ds.sample, sub_sarwing_ds.line,
        xsarsea.dir_geo_to_sample(sarwing_ds["winddir_dual"] ,sub_sarwing_ds.owiHeading),
        np.abs(wind_dual).isel(line=slice(None, None, 10), sample=slice(None, None, 10))
    )
)

hv.Image(windspeed_dual, kdims=['sample','line']).opts(title='speed and dir', clim=(0,50), cmap='jet') * vectorfield


### sarwing direction output

In [ ]:
sub_sarwing_ds = sarwing_ds.isel(line=slice(None, None, 10), sample=slice(None, None, 10))

vectorfield = hv.VectorField(
    (
        sub_sarwing_ds.sample, sub_sarwing_ds.line,
        xsarsea.dir_geo_to_sample(sub_sarwing_ds.owiWindDirection ,sub_sarwing_ds.owiHeading),
        sub_sarwing_ds.owiWindSpeed
    )
)

hv.Image(sarwing_ds.owiWindSpeed, kdims=['sample','line']).opts(title='speed and dir', clim=(0,50), cmap='jet') * vectorfield


## direction difference

In [ ]:
def normalize_angle(diff):
    return (diff + 180) % 360 - 180
    
angle_diff = normalize_angle(sarwing_ds["winddir_dual"] - sarwing_ds.owiWindDirection_Tab_dualpol_2steps)

(
    (hv.Image(sarwing_ds["winddir_dual"]).opts(title='xsarsea') + 
     hv.Image(sarwing_ds.owiWindDirection_Tab_dualpol_2steps).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,360))) +  
    hv.Image(normalize_angle(angle_diff)).opts(cmap='jet', clim=(-5,5), title='xsarsea-sarwing\nmean=%.4f std=%.4f' % (np.mean(angle_diff), np.std(angle_diff)))
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

In [ ]:
np.angle(wind_co)

In [ ]:
wphi_co = 15.0
wspd_co = 30
sol = wspd_co * np.exp(1j * np.deg2rad(wphi_co))

In [ ]:
np.rad2deg(np.angle(sol))